## SetUp

In [1]:
%AddJar file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar

Starting download from file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar
Finished download of Emiasd-Flight-Data-Analysis.jar
Using cached version of Emiasd-Flight-Data-Analysis.jar


In [2]:
import org.apache.spark.sql.SparkSession
import com.flightdelay.config.{AppConfiguration, ConfigurationLoader, ExperimentConfig}
import com.flightdelay.data.loaders.FlightDataLoader

// Env Configuration
val args: Array[String] = Array("jupyter")
implicit val configuration: AppConfiguration = ConfigurationLoader.loadConfiguration(args)
implicit val experimentConfig: ExperimentConfig = configuration.experiments(0)

val spark = SparkSession.builder()
  .config(sc.getConf)
  .config("spark.eventLog.enabled", "true")
  .config("spark.eventLog.dir", s"${configuration.common.output.basePath}/spark-events")  // ex: "file:/tmp/spark-events" ou "hdfs:///spark-events"
  .getOrCreate()

// Rendre la session Spark implicite
implicit val session = spark


args = Array(jupyter)
configuration = AppConfiguration(local,CommonConfig(42,true,debug,false,false,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201*.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/20101*.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileConfig(/home/jovyan/work/output/model)),MLFlowConfig(false,http://localhost:5555)),Stream(ExperimentConfig(Experience-local,Baseline Random ...


AppConfiguration(local,CommonConfig(42,true,debug,false,false,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201*.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/20101*.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileConfig(/home/jovyan/work/output/model)),MLFlowConfig(false,http://localhost:5555)),Stream(ExperimentConfig(Experience-local,Baseline Random ...

In [3]:
val joinedDataPath = s"${configuration.common.output.basePath}/Experience-local-base/data/join_exploded_train_prepared.parquet"
val joinedData = spark.read.parquet(joinedDataPath)

println("Joined DF Count: ", joinedData.count())


(Joined DF Count: ,117706)


joinedDataPath = /home/jovyan/work/output/Experience-local-base/data/join_exploded_train_prepared.parquet
joinedData = [CRS_ARR_TIME: string, CRS_DEP_TIME: int ... 194 more fields]


[CRS_ARR_TIME: string, CRS_DEP_TIME: int ... 194 more fields]

In [4]:
joinedData.printSchema

root
 |-- CRS_ARR_TIME: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- D1: integer (nullable = true)
 |-- D2_15: integer (nullable = true)
 |-- D2_30: integer (nullable = true)
 |-- D2_45: integer (nullable = true)
 |-- D2_60: integer (nullable = true)
 |-- D2_90: integer (nullable = true)
 |-- D3: integer (nullable = true)
 |-- D4: integer (nullable = true)
 |-- DEST_AIRPORT_ID: integer (nullable = true)
 |-- DEST_WBAN: string (nullable = true)
 |-- FL_DATE: date (nullable = true)
 |-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN_AIRPORT_ID: integer (nullable = true)
 |-- ORIGIN_WBAN: string (nullable = true)
 |-- UTC_CRS_DEP_TIME: string (nullable = true)
 |-- UTC_FL_DATE: date (nullable = true)
 |-- depHour: integer (nullable = true)
 |-- feature_arrival_time_period: string (nullable = true)
 |-- feature_departure_hour_rounded: long (nullable 

In [5]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

def countMissing(df: org.apache.spark.sql.DataFrame): org.apache.spark.sql.DataFrame = {

  val exprs = df.schema.fields.map { f =>
    val c  = f.name
    val dt = f.dataType

    val cond = dt match {
      case DoubleType | FloatType =>
        col(c).isNull || isnan(col(c))
      case StringType =>
        col(c).isNull || col(c) === ""
      case _ =>
        col(c).isNull
    }

    sum(when(cond, 1).otherwise(0)).alias(c)
  }

  df.agg(exprs.head, exprs.tail: _*)
}

// Exemple :
countMissing(joinedData).show(200, 1000, true)

countMissing: (df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


-RECORD 0-------------------------------------------------------
 CRS_ARR_TIME                                           | 0     
 CRS_DEP_TIME                                           | 0     
 CRS_ELAPSED_TIME                                       | 0     
 D1                                                     | 0     
 D2_15                                                  | 0     
 D2_30                                                  | 0     
 D2_45                                                  | 0     
 D2_60                                                  | 0     
 D2_90                                                  | 0     
 D3                                                     | 0     
 D4                                                     | 0     
 DEST_AIRPORT_ID                                        | 0     
 DEST_WBAN                                              | 0     
 FL_DATE                                                | 0     
 OP_CARRIER_AIRLINE_ID   

In [6]:
import com.flightdelay.features.pipelines.ConfigurationBasedFeatureExtractorPipeline
import org.apache.spark.ml.{Pipeline, PipelineModel, PipelineStage}


val handleInvalid = "keep"

val configPipeline = ConfigurationBasedFeatureExtractorPipeline(
  featureConfig = experimentConfig.featureExtraction,
  target = "is_delayed",
  handleInvalid = handleInvalid
)


handleInvalid = keep
configPipeline = com.flightdelay.features.pipelines.ConfigurationBasedFeatureExtractorPipeline@4272e766


com.flightdelay.features.pipelines.ConfigurationBasedFeatureExtractorPipeline@4272e766

## ConfigurationBasedFeatureExtractorPipeline

In [7]:
val preprocessed = configPipeline.preprocessBooleans(joinedData)
val stages = configPipeline.buildPipelineStages(preprocessed, experimentConfig)


[INFO] [ConfigurationBasedFeatureExtractorPipeline] Feature Distribution:
[INFO]   - StringIndexer features: 49
[INFO]     OP_CARRIER_AIRLINE_ID, destination_weather_feature_pressure_bucket_h1, destination_weather_feature_pressure_bucket_h2, destination_weather_feature_pressure_bucket_h3, origin_weather_feature_pressure_bucket_h1, ... (44 more)
[DEBUG] [ConfigurationBasedFeatureExtractorPipeline] Categorical Features Cardinality:
[DEBUG]   [ 0] OP_CARRIER_AIRLINE_ID                              :     16 distinct values
[DEBUG]   [ 1] destination_weather_feature_pressure_bucket_h1     :      5 distinct values
[DEBUG]   [ 2] destination_weather_feature_pressure_bucket_h2     :      5 distinct values
[DEBUG]   [ 3] destination_weather_feature_pressure_bucket_h3     :      5 distinct values
[DEBUG]   [ 4] origin_weather_feature_pressure_bucket_h1          :      5 distinct values
[DEBUG]   [ 5] origin_weather_feature_pressure_bucket_h2          :      5 distinct values
[DEBUG]   [ 6] origi

preprocessed = [CRS_ARR_TIME: string, CRS_DEP_TIME: int ... 194 more fields]
stages = Array(strIdx_77d0a964647f, imputer_660f83caec79, VectorAssembler: uid=vecAssembler_2fb5f683b327, handleInvalid=keep, numInputCols=111, vecIdx_63efc77a2687)


Array(strIdx_77d0a964647f, imputer_660f83caec79, VectorAssembler: uid=vecAssembler_2fb5f683b327, handleInvalid=keep, numInputCols=111, vecIdx_63efc77a2687)

In [10]:
val pipeline = new Pipeline().setStages(stages)
val preprocessedWithLabel = preprocessed.withColumn("label", col(experimentConfig.target).cast(DoubleType))
val model = pipeline.fit(preprocessedWithLabel)
val transformed = configPipeline.postProcess(model.transform(preprocessedWithLabel))

pipeline = pipeline_ff5728f625b6
preprocessedWithLabel = [CRS_ARR_TIME: string, CRS_DEP_TIME: int ... 195 more fields]
model = pipeline_ff5728f625b6
transformed = [features: vector, label: double]


[features: vector, label: double]

In [11]:
transformed.show(1,10000, true)

-RECORD 0-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 features | (111,[0,1,6,7,8,11,18,22,24,28,33,40,42,43,44,45,46,51,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,78,82,83,84,85,86,87,88,95,96,97,98,101],[9.0,2.0,2.0,2.0,33.0,2.0,5.0,1.0,2.0,2.0,1.0,6.0,1.0,1.0,2.0,1.0,1.0,3.0,10.0,10.0,22.0,9.0,15.0,15.0,89.0,65.66666666666667,114.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,3.0,5.0,3.0,3.0,3.0,1.0,2.0,2.0,4.0,2.0,2.0,2.0,1725.0,18.0,12.0,2.0,5.6,1.0]) 
 label    | 1.0                                                                                                                                                                   

In [9]:
print(transformed.count())

117728

## ConfigurationBasedFeatureExtractorPipeline - Step as Step

In [19]:
// Column naming constants
val _label = "label"
val _prefix = "indexed_"
val _ohe_prefix = "ohe_"
val _featuresVec = "featuresVec"
val _features = "features"


lastException = null
_label = label
_prefix = indexed_
_ohe_prefix = ohe_
_featuresVec = featuresVec
_features = features


features

#### Indentification des colonnes

In [20]:
import com.flightdelay.features.pipelines.ConfigurationBasedFeatureExtractorPipeline

val (stringIndexerCols, oneHotEncoderCols, numericCols) = configPipeline.groupFeaturesByTransformation(preprocessed)

stringIndexerCols = Array(OP_CARRIER_AIRLINE_ID, destination_weather_feature_pressure_bucket_h1, destination_weather_feature_pressure_bucket_h2, destination_weather_feature_pressure_bucket_h3, origin_weather_feature_pressure_bucket_h1, origin_weather_feature_pressure_bucket_h2, origin_weather_feature_pressure_bucket_h3, DEST_AIRPORT_ID, ORIGIN_AIRPORT_ID, destination_weather_feature_visibility_category_h1, destination_weather_feature_visibility_category_h2, destination_weather_feature_visibility_category_h3, origin_weather_feature_visibility_category_h1, origin_weather_feature_visibility_category_h2, origin_weather_feature_visibility_category_h3, feature_arrival_time_period, destination_w...


Array(OP_CARRIER_AIRLINE_ID, destination_weather_feature_pressure_bucket_h1, destination_weather_feature_pressure_bucket_h2, destination_weather_feature_pressure_bucket_h3, origin_weather_feature_pressure_bucket_h1, origin_weather_feature_pressure_bucket_h2, origin_weather_feature_pressure_bucket_h3, DEST_AIRPORT_ID, ORIGIN_AIRPORT_ID, destination_weather_feature_visibility_category_h1, destination_weather_feature_visibility_category_h2, destination_weather_feature_visibility_category_h3, origin_weather_feature_visibility_category_h1, origin_weather_feature_visibility_category_h2, origin_weather_feature_visibility_category_h3, feature_arrival_time_period, destination_w...

In [21]:
println("--> stringIndexerCols")
stringIndexerCols.zipWithIndex.foreach { case (value, idx) =>
  println(s"[$idx] $value")
}

--> stringIndexerCols
[0] OP_CARRIER_AIRLINE_ID
[1] destination_weather_feature_pressure_bucket_h1
[2] destination_weather_feature_pressure_bucket_h2
[3] destination_weather_feature_pressure_bucket_h3
[4] origin_weather_feature_pressure_bucket_h1
[5] origin_weather_feature_pressure_bucket_h2
[6] origin_weather_feature_pressure_bucket_h3
[7] DEST_AIRPORT_ID
[8] ORIGIN_AIRPORT_ID
[9] destination_weather_feature_visibility_category_h1
[10] destination_weather_feature_visibility_category_h2
[11] destination_weather_feature_visibility_category_h3
[12] origin_weather_feature_visibility_category_h1
[13] origin_weather_feature_visibility_category_h2
[14] origin_weather_feature_visibility_category_h3
[15] feature_arrival_time_period
[16] destination_weather_extracted_codes_h1
[17] destination_weather_extracted_codes_h2
[18] destination_weather_extracted_codes_h3
[19] origin_weather_extracted_codes_h1
[20] origin_weather_extracted_codes_h2
[21] origin_weather_extracted_codes_h3
[22] destination_

In [22]:
println("--> oneHotEncoderCols")
oneHotEncoderCols.zipWithIndex.foreach { case (value, idx) =>
  println(s"[$idx] $value")
}

--> oneHotEncoderCols


In [23]:
println("--> numericCols")
numericCols.zipWithIndex.foreach { case (value, idx) =>
  println(s"[$idx] $value")
}

--> numericCols
[0] destination_weather_weather_hazard_level_h1
[1] destination_weather_weather_hazard_level_h2
[2] destination_weather_weather_hazard_level_h3
[3] origin_weather_weather_hazard_level_h1
[4] origin_weather_weather_hazard_level_h2
[5] origin_weather_weather_hazard_level_h3
[6] destination_weather_feature_weather_severity_index_h1
[7] destination_weather_feature_weather_severity_index_h2
[8] destination_weather_feature_weather_severity_index_h3
[9] origin_weather_feature_weather_severity_index_h1
[10] origin_weather_feature_weather_severity_index_h2
[11] origin_weather_feature_weather_severity_index_h3
[12] destination_weather_RelativeHumidity_h1
[13] destination_weather_RelativeHumidity_h2
[14] destination_weather_RelativeHumidity_h3
[15] origin_weather_RelativeHumidity_h1
[16] origin_weather_RelativeHumidity_h2
[17] origin_weather_RelativeHumidity_h3
[18] CRS_ELAPSED_TIME
[19] destination_weather_feature_flight_category_ordinal_h1
[20] destination_weather_feature_flight

#### Pipeline

In [24]:
var stages = Array.empty[PipelineStage]

stages = Array()


Array()

In [25]:
import org.apache.spark.ml.feature._

val target = "is_delayed"
val labelIndexer = new StringIndexer()
  .setInputCol(target)
  .setOutputCol(_label)
  .setHandleInvalid("skip")  // Always skip for label, regardless of handleInvalid parameter
stages = stages :+ labelIndexer

target = is_delayed
labelIndexer = strIdx_0052e92296e4
stages = [Lorg.apache.spark.ml.PipelineStage;@5805f2f3


[Lorg.apache.spark.ml.PipelineStage;@5805f2f3

In [26]:
if (stringIndexerCols.nonEmpty) {
  val categoricalIndexer = new StringIndexer()
    .setInputCols(stringIndexerCols)
    .setOutputCols(stringIndexerCols.map(_prefix + _))
    .setHandleInvalid(handleInvalid)
  stages = stages :+ categoricalIndexer
}

In [27]:
import com.flightdelay.utils.DebugUtils._

if (numericCols.nonEmpty) {
  val imputer = new Imputer()
    .setInputCols(numericCols)
    .setOutputCols(numericCols.map("imputed_" + _))
    .setStrategy("mean")  // Replace NaN with mean (alternatives: "median", "mode")
    .setMissingValue(Double.NaN)
  stages = stages :+ imputer

  info(s"  - Added Imputer for ${numericCols.length} numeric columns (strategy: mean)")
}

[INFO]   - Added Imputer for 74 numeric columns (strategy: mean)


In [28]:
// Stage 5: VectorAssembler to combine all features
val imputedNumericCols = if (numericCols.nonEmpty) numericCols.map("imputed_" + _) else Array.empty[String]
val allInputCols = stringIndexerCols.map(_prefix + _) ++
                   imputedNumericCols

imputedNumericCols = Array(imputed_destination_weather_weather_hazard_level_h1, imputed_destination_weather_weather_hazard_level_h2, imputed_destination_weather_weather_hazard_level_h3, imputed_origin_weather_weather_hazard_level_h1, imputed_origin_weather_weather_hazard_level_h2, imputed_origin_weather_weather_hazard_level_h3, imputed_destination_weather_feature_weather_severity_index_h1, imputed_destination_weather_feature_weather_severity_index_h2, imputed_destination_weather_feature_weather_severity_index_h3, imputed_origin_weather_feature_weather_severity_index_h1, imputed_origin_weather_feature_weather_severity_index_h2, imputed_origin_weather_feature_weather_severity_index_h3, imputed_destination_weather_RelativeHumidity_h1, imputed_destination_weather_RelativeHumi...


Array(imputed_destination_weather_weather_hazard_level_h1, imputed_destination_weather_weather_hazard_level_h2, imputed_destination_weather_weather_hazard_level_h3, imputed_origin_weather_weather_hazard_level_h1, imputed_origin_weather_weather_hazard_level_h2, imputed_origin_weather_weather_hazard_level_h3, imputed_destination_weather_feature_weather_severity_index_h1, imputed_destination_weather_feature_weather_severity_index_h2, imputed_destination_weather_feature_weather_severity_index_h3, imputed_origin_weather_feature_weather_severity_index_h1, imputed_origin_weather_feature_weather_severity_index_h2, imputed_origin_weather_feature_weather_severity_index_h3, imputed_destination_weather_RelativeHumidity_h1, imputed_destination_weather_RelativeHumi...

In [29]:
println("--> allInputCols")
allInputCols.zipWithIndex.foreach { case (value, idx) =>
  println(s"[$idx] $value")
}

--> allInputCols
[0] indexed_OP_CARRIER_AIRLINE_ID
[1] indexed_destination_weather_feature_pressure_bucket_h1
[2] indexed_destination_weather_feature_pressure_bucket_h2
[3] indexed_destination_weather_feature_pressure_bucket_h3
[4] indexed_origin_weather_feature_pressure_bucket_h1
[5] indexed_origin_weather_feature_pressure_bucket_h2
[6] indexed_origin_weather_feature_pressure_bucket_h3
[7] indexed_DEST_AIRPORT_ID
[8] indexed_ORIGIN_AIRPORT_ID
[9] indexed_destination_weather_feature_visibility_category_h1
[10] indexed_destination_weather_feature_visibility_category_h2
[11] indexed_destination_weather_feature_visibility_category_h3
[12] indexed_origin_weather_feature_visibility_category_h1
[13] indexed_origin_weather_feature_visibility_category_h2
[14] indexed_origin_weather_feature_visibility_category_h3
[15] indexed_feature_arrival_time_period
[16] indexed_destination_weather_extracted_codes_h1
[17] indexed_destination_weather_extracted_codes_h2
[18] indexed_destination_weather_extrac

In [30]:
val vectorAssembler = new VectorAssembler()
  .setInputCols(allInputCols)
  .setOutputCol(_featuresVec)
  .setHandleInvalid(handleInvalid)
stages = stages :+ vectorAssembler

vectorAssembler = VectorAssembler: uid=vecAssembler_19f777c1fade, handleInvalid=keep, numInputCols=123
stages = [Lorg.apache.spark.ml.PipelineStage;@5e293375


[Lorg.apache.spark.ml.PipelineStage;@5e293375

In [31]:
// Stage 6: VectorIndexer pour RandomForest (remplace StandardScaler)
val vectorIndexer = new VectorIndexer()
  .setInputCol(_featuresVec)
  .setOutputCol(_features)          // "features" final pour le modèle
  .setMaxCategories(experimentConfig.featureExtraction.maxCategoricalCardinality)            // à ajuster selon ton cas
stages = stages :+ vectorIndexer

vectorIndexer = vecIdx_61eadc1219f4
stages = [Lorg.apache.spark.ml.PipelineStage;@70b6c871


[Lorg.apache.spark.ml.PipelineStage;@70b6c871

In [32]:
val pipeline = new Pipeline().setStages(stages)
val model = pipeline.fit(preprocessed)
val transformed = model.transform(preprocessed)

pipeline = pipeline_03e135d91092
model = pipeline_03e135d91092
transformed = [CRS_ARR_TIME: string, CRS_DEP_TIME: int ... 290 more fields]


[CRS_ARR_TIME: string, CRS_DEP_TIME: int ... 290 more fields]

In [33]:
transformed.select(_featuresVec, _features, target, _label).show(1, 1000, true)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 featuresVec | (123,[0,7,8,15,41,42,44,45,46,47,48,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,86,87,88,89,90,91,92,99,100,101,102,103,104,105,106,107,108,109,110],[8.0,9.0,13.0,2.0,3.0,1.0,3.0,3.0,3.0,1.0,6.0,2.5999999999999996,3.0,3.0,1.7999999999999998,1.7999999999999998,1.7999999999999998,82.0,76.0,74.0,18.0,17.0,16.0,304.0,2.0,

In [43]:
val postProcessed = configPipeline.postProcess(transformed)
postProcessed.show(1, 1000, true)

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 features | (93,[0,7,8,15,29,30,32,33,34,35,36,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,74,75,76,77,78,79,80,81,82,83,84,85,86],[8.0,9.0,13.0,2.0,3.0,1.0,3.0,3.0,3.0,1.0,6.0,7.0,10.0,8.0,5.0,7.0,7.0,82.0,76.0,74.0,18.0,17.0,16.0,304.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,3.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,1326.0,9.97918413653628,10.14760566158702,10.408580234639093,10.74273406941625,11.12462477486492,11.047547093784628]) 
 label    | 1.0                                                                 

postProcessed = [features: vector, label: double]


[features: vector, label: double]

In [6]:
print(configuration)

AppConfiguration(local,CommonConfig(42,true,debug,false,false,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201*.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/20101*.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileConfig(/home/jovyan/work/output/model)),MLFlowConfig(false,http://localhost:5555)),Stream(ExperimentConfig(Experience-local,Baseline Random Forest with hyperparameters tuning,true,is_delayed,FeatureExtractionConfig(feature_selection,D2_60,60,0.95,true,true,3,3,50,skip,Some(Map(OP_CARRIER_AIRLINE_ID -> FeatureTransformationConfig(StringIndexer), DEST_AIRPORT_ID -> FeatureTransformationConfig(StringIndexer), ORIGIN_AIRPORT_ID -> FeatureTransformationConfig(StringIndexer), CRS_ELAPSED_TIME -> FeatureTransformationConfig(None), feature_arrival_time_period -> FeatureTransformationConfig(StringIndexer), CRS_DEP_TIME -

In [34]:
import org.apache.spark.ml.attribute.AttributeGroup
import org.apache.spark.ml.linalg.Vector

// Nom de ta colonne de features
val featuresColName = _featuresVec   // "featuresVec"

// 1) Récupérer le groupe d’attributs
val fg = AttributeGroup.fromStructField(transformed.schema(featuresColName))

// 2) Extraire la liste des noms de features dans l’ordre des indices
val featureNames: Array[String] =
  if (fg.attributes.isDefined) {
    fg.attributes.get.map(attr => attr.name.getOrElse("")).toArray
  } else {
    // fallback si jamais les métadonnées sont absentes
    Array.tabulate(fg.size)(i => s"feature_$i")
  }

// 3) Afficher le mapping complet index -> featureName
println(s"Nombre de features = ${featureNames.length}")
featureNames.zipWithIndex.foreach { case (name, idx) =>
  println(f"$idx%3d -> $name")
}

Nombre de features = 123
  0 -> indexed_OP_CARRIER_AIRLINE_ID
  1 -> indexed_destination_weather_feature_pressure_bucket_h1
  2 -> indexed_destination_weather_feature_pressure_bucket_h2
  3 -> indexed_destination_weather_feature_pressure_bucket_h3
  4 -> indexed_origin_weather_feature_pressure_bucket_h1
  5 -> indexed_origin_weather_feature_pressure_bucket_h2
  6 -> indexed_origin_weather_feature_pressure_bucket_h3
  7 -> indexed_DEST_AIRPORT_ID
  8 -> indexed_ORIGIN_AIRPORT_ID
  9 -> indexed_destination_weather_feature_visibility_category_h1
 10 -> indexed_destination_weather_feature_visibility_category_h2
 11 -> indexed_destination_weather_feature_visibility_category_h3
 12 -> indexed_origin_weather_feature_visibility_category_h1
 13 -> indexed_origin_weather_feature_visibility_category_h2
 14 -> indexed_origin_weather_feature_visibility_category_h3
 15 -> indexed_feature_arrival_time_period
 16 -> indexed_destination_weather_extracted_codes_h1
 17 -> indexed_destination_weather_extr

featuresColName = featuresVec
fg = {"ml_attr":{"attrs":{"numeric":[{"idx":49,"name":"imputed_destination_weather_weather_hazard_level_h1"},{"idx":50,"name":"imputed_destination_weather_weather_hazard_level_h2"},{"idx":51,"name":"imputed_destination_weather_weather_hazard_level_h3"},{"idx":52,"name":"imputed_origin_weather_weather_hazard_level_h1"},{"idx":53,"name":"imputed_origin_weather_weather_hazard_level_h2"},{"idx":54,"name":"imputed_origin_weather_weather_hazard_level_h3"},{"idx":55,"name":"imputed_destination_weather_feature_weather_severity_index_h1"},{"idx":56,"name":"imputed_destination_weather_feature_weather_severity_index_h2"},{"...


{"ml_attr":{"attrs":{"numeric":[{"idx":49,"name":"imputed_destination_weather_weather_hazard_level_h1"},{"idx":50,"name":"imputed_destination_weather_weather_hazard_level_h2"},{"idx":51,"name":"imputed_destination_weather_weather_hazard_level_h3"},{"idx":52,"name":"imputed_origin_weather_weather_hazard_level_h1"},{"idx":53,"name":"imputed_origin_weather_weather_hazard_level_h2"},{"idx":54,"name":"imputed_origin_weather_weather_hazard_level_h3"},{"idx":55,"name":"imputed_destination_weather_feature_weather_severity_index_h1"},{"idx":56,"name":"imputed_destination_weather_feature_weather_severity_index_h2"},{"...